# Imports and Setups

In [1]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

### Setup the `device` variable.

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Download and Prepare Dataset

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

9913344it [00:01, 6368656.82it/s]                             


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



29696it [00:00, 29662789.14it/s]         

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



1649664it [00:03, 540098.64it/s]                             


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



5120it [00:00, ?it/s]                   

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw




C:\Users\reese\miniconda3\envs\final-year-project\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

# Define Model Architecture

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)     # output: 26*26*32
        self.conv2 = nn.Conv2d(32, 64, 3, 1)    # output: 24*24*64
        self.conv3 = nn.Conv2d(64, 128, 3, 1)   # output: 22*22*128
        self.maxpool1 = nn.MaxPool2d(2,2)       # output: 11*11*128
        self.fc1 = nn.Linear(15488, 7744)       # in:15488 out:7744
        self.fc2 = nn.Linear(7744, 128)         # in:7744 out:128
        self.fc3 = nn.Linear(128, 10)           # in:128  out:10

    def forward(self, x):
        ## Conv 1st Block
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.maxpool1(x)
        #output: 11*11*64

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

# Training Loop

In [6]:
def train(model, device, train_loader, optimizer, epoch, steps_per_epoch=20):
  # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.train()
  train_total = 0
  train_correct = 0

  # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
  for batch_idx, (data, target) in enumerate(train_loader, start=0):
    if batch_idx > steps_per_epoch:
      break
    # Load the input features and labels from the training dataset
    data, target = data.to(device), target.to(device)
    
    # Reset the gradients to 0 for all learnable weight parameters
    optimizer.zero_grad()
    
    # Forward pass: Pass image data from training dataset, make predictions about class image belongs to (0-9 in this case)
    output = model(data)
    
    # Define our loss function, and compute the loss
    loss = F.nll_loss(output, target)

    scores, predictions = torch.max(output.data, 1)
    train_total += target.size(0)
    train_correct += int(sum(predictions == target))
            
    # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
    loss.backward()
    
    # Update the neural network weights
    optimizer.step()

  acc = round((train_correct / train_total) * 100, 2)
  print('Epoch [{}], Loss: {}, Accuracy: {}'.format(epoch, loss.item(), acc), end='')
  

# Testing Loop

In [7]:
def test(model, device, test_loader, classes):
  # Switch model to evaluation mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.eval()
  test_loss = 0
  test_total = 0
  test_correct = 0

  example_images = []
  with torch.no_grad():
      for data, target in test_loader:
          # Load the input features and labels from the test dataset
          data, target = data.to(device), target.to(device)
          
          # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
          output = model(data)
          
          # Compute the loss sum up batch loss
          test_loss += F.nll_loss(output, target, reduction='sum').item()
          
          scores, predictions = torch.max(output.data, 1)
          test_total += target.size(0)
          test_correct += int(sum(predictions == target))
          
          # WandB – Log images in your test dataset automatically, along with predicted and true labels by passing pytorch tensors with image data into wandb.Image
          # example_images.append(wandb.Image(
          #     data[0], caption="Pred: {} Truth: {}".format(classes[pred[0].item()], classes[target[0]])))
  acc = round((test_correct / test_total) * 100, 2)
  print(' Test_loss: {}, Test_accuracy: {}'.format(test_loss/test_total, acc))


# Train

In [8]:
net = Net().to(device)
print(net)

optimizer = optim.Adam(net.parameters())

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=15488, out_features=7744, bias=True)
  (fc2): Linear(in_features=7744, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [9]:

for epoch in range(10):
  train(net, device, trainloader, optimizer, epoch)
  test(net, device, testloader, classes)

print('Finished Training')

C:\Users\reese\miniconda3\envs\final-year-project\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [0], Loss: 0.6975080370903015, Accuracy: 39.29 Test_loss: 0.9074764924049378, Test_accuracy: 66.58
Epoch [1], Loss: 0.2872771620750427, Accuracy: 85.04 Test_loss: 0.28694743301868436, Test_accuracy: 91.11
Epoch [2], Loss: 0.2117382138967514, Accuracy: 91.44 Test_loss: 0.26369774926900863, Test_accuracy: 91.96
Epoch [3], Loss: 0.06233030557632446, Accuracy: 93.38 Test_loss: 0.13696479143723844, Test_accuracy: 95.9
Epoch [4], Loss: 0.21373508870601654, Accuracy: 94.57 Test_loss: 0.12290838470235467, Test_accuracy: 96.43
Epoch [5], Loss: 0.09459520876407623, Accuracy: 96.5 Test_loss: 0.13406563600748778, Test_accuracy: 95.94
Epoch [6], Loss: 0.18634946644306183, Accuracy: 95.83 Test_loss: 0.10705296227410435, Test_accuracy: 96.53
Epoch [7], Loss: 0.10909657180309296, Accuracy: 96.43 Test_loss: 0.08392147594615817, Test_accuracy: 97.34
Epoch [8], Loss: 0.17012055218219757, Accuracy: 97.4 Test_loss: 0.10885600378587842, Test_accuracy: 96.55
Epoch [9], Loss: 0.21047110855579376, Accura